In [ ]:
import sqlite3
from langgraph.graph import StateGraph, START, END
from typing_extensions import TypedDict
from langchain.chat_models import init_chat_model
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages, MessagesState
from typing import Annotated
from langgraph.prebuilt import ToolNode, tools_condition
from langchain_core.tools import tool
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.types import interrupt

llm = init_chat_model("openai:gpt-4o-mini")

conn = sqlite3.connect("memory.db", check_same_thread=False)

config = {
    "configurable": {
        "thread_id": "4",
    },
}

# llm.invoke([{
#     "role": "user",
#     "content": "hi!"
# }])

In [8]:
# class State(TypedDict):
#     messages: Annotated[list[AnyMessage], add_messages]

class State(MessagesState):
    pass

graph_builder = StateGraph(State)

In [ ]:
@tool
def get_human_feedback(poem: str) -> str:
    """
        Asks the user for feedback on the poem
        Use this before returning the final response
    """
    feedback = interrupt(f"Please provide feedback on the poem\n{poem}")
    return feedback


llm_with_tools = llm.bind_tools(tools=[get_human_feedback])


def chatbot(state: State):
    response = llm_with_tools.invoke(f"""
    당신은 시를 만드는 전문가 입니다. 

    'get_human_feedback' 함수를 사용해서 시에 대한 피드백을 받으세요.

    긍정적인 피드백을 받은 후에만 완성된 만들어주세요.

    항상 피드백을 받으세요.

    이건 지난 대화 기록 입니다 : 
    {state["messages"]}

    """)
    return {
        "messages": [response]
    }


In [10]:
tool_node = ToolNode(
    tools=[
        get_human_feedback,
    ]
)

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", tool_node)

graph_builder.add_edge(START, "chatbot")
graph_builder.add_conditional_edges("chatbot", tools_condition)
graph_builder.add_edge("tools", "chatbot")
# graph_builder.add_edge("chatbot", END)

graph = graph_builder.compile(
    checkpointer=SqliteSaver(conn)
)

In [11]:
result = graph.invoke(
    {
        "messages": [
            {
                "role": "user",
                "content": "제주도 여행을 주제로 짧은 시를 만들어줘. 음율(라임)을 최대한 살려서 감성적이고 멋지게 만들어줘. "
            }
        ]
    },
    config=config,
)

In [12]:
for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

제주도 여행을 주제로 짧은 시를 만들어줘. 음율(라임)을 최대한 살려서 감성적이고 멋지게 만들어줘. 
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_VAlRDV5ExXnsAP1Tn4LcbVCC)
 Call ID: call_VAlRDV5ExXnsAP1Tn4LcbVCC
  Args:
    poem: 푸른 바다에 물결 춤추고,
제주 햇살에 마음이 부풀어.
한라산 높이, 구름을 가르며,
여행길에 행복이 물들어.

오름을 넘어, 소리 없는 고요,
바람에 실린, 제주 꿈이 흘러.
섭지코지의 노을 아래,
추억 속에 당신과 나의 미소.


In [16]:
snapshot = graph.get_state(config)

snapshot.next

('tools',)

In [14]:
from langgraph.types import Command

use_cmd = Command(
    resume="좋아보여요"
)

result = graph.invoke(
    use_cmd,
    config=config,
)

for message in result["messages"]:
    message.pretty_print()

================================ Human Message =================================

제주도 여행을 주제로 짧은 시를 만들어줘. 음율(라임)을 최대한 살려서 감성적이고 멋지게 만들어줘. 
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_VAlRDV5ExXnsAP1Tn4LcbVCC)
 Call ID: call_VAlRDV5ExXnsAP1Tn4LcbVCC
  Args:
    poem: 푸른 바다에 물결 춤추고,
제주 햇살에 마음이 부풀어.
한라산 높이, 구름을 가르며,
여행길에 행복이 물들어.

오름을 넘어, 소리 없는 고요,
바람에 실린, 제주 꿈이 흘러.
섭지코지의 노을 아래,
추억 속에 당신과 나의 미소.
================================= Tool Message =================================
Name: get_human_feedback

좋아보여요
================================== Ai Message ==================================
Tool Calls:
  get_human_feedback (call_ljthvrKrrpzpueO463DIyNhk)
 Call ID: call_ljthvrKrrpzpueO463DIyNhk
  Args:
    poem: 푸른 바다에 물결 춤추고,
제주 햇살에 마음이 부풀어.
한라산 높이, 구름을 가르며,
여행길에 행복이 물들어.

오름을 넘어, 소리 없는 고요,
바람에 실린, 제주 꿈이 흘러.
섭지코지의 노을 아래,
추억 속에 당신과 나의 미소.
